# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [5]:
# all import statements needed for the project, for example:

import math
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import re
import pyarrow.parquet as pq
import geopandas as gpd
import os
from math import sin, cos, sqrt, atan2, radians

In [6]:
# any general notebook setup, like log formatting
import warnings
warnings.filterwarnings("ignore")

In [51]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [8]:
def calculate_distance(start_lat, start_lon, end_lat, end_lon):
    R = 6373.0

    lat1 = radians(start_lat)
    lon1 = radians(start_lon)
    lat2 = radians(end_lat)
    lon2 = radians(end_lon)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return round(distance, 2)

    print("Result:", distance)
    print("Should be:", 278.546, "km")

In [9]:
def add_distance_column(df):
    distance = []
    
    for i in df.index:
        estimated_distance = calculate_distance(df["pickup_latitude"][i], df["pickup_longitude"][i], df["dropoff_latitude"][i], df["dropoff_longitude"][i])
        distance.append(estimated_distance)
        
    df["calculated_distance"] = distance
    
    return df


### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [10]:
def find_taxi_csv_urls():
    TAXI_URL = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
    
    response = requests.get(TAXI_URL)
    html = response.content
    data = []
    soup = bs4.BeautifulSoup(html, "html.parser")
    for link in soup("a"):
        name = link.get("title")
        date = link.get("href")
        date_pattern = r'201[012345]|2009'
        if name == "Yellow Taxi Trip Records" and re.search(date_pattern, date):
            if not re.search(r'2015\-0[789]|2015\-1[012]', date):
                data.append(link.get("href"))
    return data

In [11]:
def get_lat_lon_from_loc():
    find_lat_lon = gpd.read_file("taxi_zones/taxi_zones.shp")
  
    find_lat_lon = find_lat_lon.to_crs(4326)
    lon = find_lat_lon.centroid.x 
    lat = find_lat_lon.centroid.y
    find_lat_lon["lon"] = lon
    find_lat_lon["lat"] = lat

    return find_lat_lon
    

In [12]:
def filter_lat_lon(df):
    lon_border = [-74.242330, -73.717047]
    lat_border = [40.560445, 40.908524]
    
    deleted_row = []
    
    for i in df.index:
        if df["pickup_longitude"][i] < lon_border[0] or df["pickup_longitude"][i] > lon_border[1] or df["dropoff_longitude"][i] < lon_border[0] or df["dropoff_longitude"][i] > lon_border[1]:
            deleted_row.append(i)
            
        elif df["pickup_latitude"][i] < lat_border[0] or df["pickup_latitude"][i] > lat_border[1] or df["dropoff_latitude"][i] < lat_border[0] or df["dropoff_latitude"][i] > lat_border[1]:
            deleted_row.append(i)
            
            
    df = df.drop(labels = deleted_row, axis=0)
    
    return df.reset_index()
    

In [42]:
def get_and_clean_month_taxi_data(url):
    
    # Check if we already have the data in local, otherwise download it
    file_name = url[-31:-1]
    
    for i in os.listdir('./taxidata'):
        if i == file_name:
            df = pd.read_parquet(f"taxidata/{file_name}", engine='pyarrow')
            return df
    
    
    # Download and preprocessing data
    find_lat_lon = get_lat_lon_from_loc() #get the lat-lon form .shp file
    
    # Read the data from url
    response = requests.get(url, stream=True)
    with open(f"taxidata/{url[-31:-1]}", "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    df = pd.read_parquet(f"taxidata/{url[-31:-1]}", engine='pyarrow') #Reading the data
    df = df.sample(n=2564).reset_index() #Take 2564 random sample
    
    # Rename the columns
    if 'Start_Lon' in df.columns : #2009
        df = df[["Trip_Pickup_DateTime", "Trip_Distance", "Start_Lon", "Start_Lat", "End_Lon", "End_Lat"]]
        df.rename(columns={"Trip_Pickup_DateTime": 'date_time', "Trip_Distance": 'trip_distance', "Start_Lon": 'pickup_longitude', "Start_Lat": 'pickup_latitude', "End_Lon": 'dropoff_longitude', "End_Lat": 'dropoff_latitude'}, inplace=True)
        
    elif 'pickup_longitude' in df.columns : #2010
        df = df[["pickup_datetime", "trip_distance", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"]]
        df.rename(columns={"pickup_datetime": 'date_time', "trip_distance": 'trip_distance'}, inplace=True)

    else:     
        if "tpep_pickup_datetime" in df.columns : #2011-2014
            df = df[["tpep_pickup_datetime", "trip_distance", "PULocationID", "DOLocationID"]]
            df.rename(columns={"tpep_pickup_datetime": 'date_time'}, inplace=True)
        
        else: df = df[["date_time", "trip_distance", "PULocationID", "DOLocationID"]] #2015
            
            
        # Finding the lat-lon using shp file
        start_lon = []
        start_lat = []
        end_lon = []
        end_lat = []
        
        for i in range(len(df["PULocationID"])):
            start_point = df["PULocationID"][i]
            end_point = df["DOLocationID"][i]
            
            if df["PULocationID"][i] < 264 and df["DOLocationID"][i] < 264: #Filter for NYC Area only
                index_location = find_lat_lon[find_lat_lon["LocationID"] == start_point].index.values[0] 
                start_lon.append(float(find_lat_lon["lon"][index_location]))
                start_lat.append(float(find_lat_lon["lat"][index_location]))
                
                index_location = find_lat_lon[find_lat_lon["LocationID"] == end_point].index.values[0] 
                end_lon.append(float(find_lat_lon["lon"][index_location]))
                end_lat.append(float(find_lat_lon["lat"][index_location]))
                
            else: # Area outside NYC
                start_lon.append(0)
                start_lat.append(0)
                end_lon.append(0)
                end_lat.append(0)
            
                      
        df["pickup_longitude"] = start_lon
        df["pickup_latitude"] = start_lat
        df["dropoff_longitude"] = end_lon
        df["dropoff_latitude"] = end_lat
        
        df = df.drop(["PULocationID", "DOLocationID"], axis=1)
        
    # Filter the lat-lon between (40.560445, -74.242330) and (40.908524, -73.717047)
    df = filter_lat_lon(df)
    
    # Calculate distance and add calculated_distance column
    df = add_distance_column(df)
    
    # Re-save the file
    df.to_parquet(f"taxidata/{url[-31:-1]}")
    
    return df            
    

In [45]:
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2011-02.parquet'
get_and_clean_month_taxi_data(url)

,index,date_time,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,calculated_distance
0,0,2011-02-23 14:36:57,1.40,-73.981532,40.773633,-73.984196,40.759818,1.55
1,1,2011-02-13 03:13:52,4.00,-73.985937,40.727620,-73.981532,40.773633,5.13
2,2,2011-02-27 01:05:00,4.01,-73.934829,40.754243,-73.981532,40.773633,4.49
3,3,2011-02-09 05:54:25,1.30,-73.976495,40.740439,-73.999917,40.748428,2.16
4,4,2011-02-14 19:57:05,4.30,-73.951010,40.778766,-73.997380,40.728340,6.84
...,...,...,...,...,...,...,...,...
2457,2559,2011-02-08 12:58:00,0.65,-73.984196,40.759818,-73.977569,40.764421,0.76
2458,2560,2011-02-09 11:45:00,0.76,-73.988787,40.753513,-74.004015,40.753309,1.28
2459,2561,2011-02-13 13:14:51,2.80,-74.002875,40.734576,-73.977698,40.758028,3.36
2460,2562,2011-02-04 13:32:00,1.74,-73.978632,40.783961,-73.965635,40.768615,2.03


In [46]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_urls:
        # Get and clean the data from local or url
        dataframe = get_and_clean_month_taxi_data(csv_url)
        
        # Put all the dataframe into a list
        all_taxi_dataframes.append(dataframe)
        
    # Create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes, ignore_index=True)
    return taxi_data

In [47]:
get_and_clean_taxi_data()

,index,date_time,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,calculated_distance
0,0,2015-01-14 19:55:44,3.80,-73.978492,40.747746,-73.951010,40.778766,4.16
1,1,2015-01-16 18:56:41,1.00,-73.985937,40.727620,-74.007486,40.726290,1.82
2,2,2015-01-23 13:00:50,1.20,-73.978492,40.747746,-73.992438,40.748497,1.18
3,3,2015-01-10 01:57:55,1.64,-73.985937,40.727620,-73.976495,40.740439,1.63
4,4,2015-01-01 14:34:01,3.78,-73.972356,40.756688,-74.004015,40.753309,2.69
...,...,...,...,...,...,...,...,...
195114,2559,2009-12-31 23:35:00,1.33,-73.974645,40.749850,-73.973650,40.762767,1.44
195115,2560,2009-12-02 19:20:33,1.40,-73.978127,40.788382,-73.987560,40.770654,2.13
195116,2561,2009-12-31 23:16:27,1.10,-74.006387,40.747004,-73.995241,40.759851,1.71
195117,2562,2009-12-04 01:06:00,1.74,-74.005102,40.746512,-73.981785,40.749378,1.99


### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [55]:
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(csv_file)
    df.drop(df.columns[[0, 1, 2, 8]], axis=1,inplace=True)
    return df

In [58]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [60]:
def clean_month_weather_data_hourly(csv_file):
    df2 = pd.read_csv('weatherdata/' + csv_file)
    df2 = df2[['DATE', 'LATITUDE', 'LONGITUDE', 'HourlyPrecipitation', 'HourlyWindSpeed']]
    return df2

In [61]:
def clean_month_weather_data_daily(csv_file):
    df3 = pd.read_csv('weatherdata/' + csv_file)
    df3 = df3[['DATE', 'LATITUDE', 'LONGITUDE', 'DailyAverageWindSpeed']]
    return df3

In [62]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["2009_weather.csv", "2010_weather.csv", "2011_weather.csv", "2012_weather.csv", "2013_weather.csv", "2014_weather.csv", "2015_weather.csv"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [677]:
import sqlite3
connection = sqlite3.connect("final_project.db")
connection

In [302]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    DATE TIMESTAMP,
    HourlyPrecipitation FLOAT,
    HourlyWindSpeed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    DATE TIMESTAMP,
    DailyPrecipitation FLOAT,
    DailyAverageWindSpeed FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips
(
    id INTEGER PRIMARY KEY,
    tpep_pickup_datetime TIMESTAMP,
    tpep_dropoff_datetime TIMESTAMP,
    trip_distance FLOAT,
    total_amount FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    fare_amount FLOAT,
    pickup_datetime TIMESTAMP,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    trip_distance FLOAT
);
"""

In [678]:
# create that required schema.sql file
with open('schema.sql', "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with connection:
    connection.execute(HOURLY_WEATHER_SCHEMA)
with connection:
    connection.execute(DAILY_WEATHER_SCHEMA)
with connection:
    connection.execute(TAXI_TRIPS_SCHEMA)
with connection:
    connection.execute(UBER_TRIPS_SCHEMA)

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [684]:
def write_dataframes_to_table(table_to_df_dict):
    for key, value in table_to_df_dict.items():
        value.to_sql(name=key, con=connection, if_exists='append', index=False)
    
    return 'Success adding data'

In [880]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data
    #"uber_trips": uber_data,
    #"hourly_weather": hourly_data
    #"daily_weather": daily_data
}

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [18]:
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)
    
    return f'Succes generate {outfile}'

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [1]:
#Q1) For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? 
#The result should have 24 bins.

QUERY_1 = """
SELECT 
    DISTINCT strftime('%H', tpep_pickup_datetime) AS time,
    COUNT (*)
FROM taxi_trips
WHERE tpep_pickup_datetime between '2009-01-01' AND '2015-06-30'
GROUP BY time
"""

In [ ]:
# TOBEDELETED Read Data
with connection:
    result = connection.execute(QUERY_1)

for row in result:
    print(row)

In [ ]:
write_query_to_file(QUERY_1, "question_1.sql")

In [ ]:
#Q2) For the same time frame, what day of the week was the most popular to take an uber? 
#The result should have 7 bins.

QUERY_2 = """
SELECT  case cast (strftime('%w', pickup_datetime) as integer)
  when 0 then 'Sunday'
  when 1 then 'Monday'
  when 2 then 'Tuesday'
  when 3 then 'Wednesday'
  when 4 then 'Thursday'
  when 5 then 'Friday'
  else 'Saturday' end as day,
  COUNT(*)
FROM uber_trips
GROUP BY day
ORDER BY cast (strftime('%w', pickup_datetime) as integer) ASC
"""

# WHERE tpep_pickup_datetime between '2009-01-01' AND '2015-06-30'

In [ ]:
# TOBEDELETED Read Data
with connection:
    result = connection.execute(QUERY_2)

for row in result:
    print(row)

In [ ]:
write_query_to_file(QUERY_2, "question_2.sql")

In [ ]:
#Q3) What is the 95% percentile of distance traveled for all hired trips during July 2013?
# TBD! FILTER DATE BLM, MASIH BINGUNG MAU YG DI OERCENTILE

QUERY_3 = """
SELECT 
    trip_distance,
    percent_rank
FROM (
    SELECT
        date
        trip_distance,
        PERCENT_RANK() OVER (ORDER BY trip_distance DESC) percent_rank
    FROM (
        SELECT date(tpep_pickup_datetime) as date, trip_distance
        FROM taxi_trips
        UNION ALL
        SELECT date(pickup_datetime) as date, trip_distance
        FROM uber_trips)
    WHERE date between '2013-07-01' AND '2013-07-31'
    )
WHERE percent_rank >= 0.95
"""

In [ ]:
#Q3) What is the 95% percentile of distance traveled for all hired trips during July 2013?
# TBD! FILTER DATE BLM, MASIH BINGUNG MAU YG DI OERCENTILE

QUERY_3 = """

    SELECT
        trip_distance,
        PERCENT_RANK() OVER (ORDER BY trip_distance DESC) percent_rank
    FROM (
        SELECT date(tpep_pickup_datetime) as date, trip_distance
        FROM taxi_trips
        WHERE date between '2013-07-01' AND '2013-07-31'
        UNION ALL
        SELECT date(pickup_datetime) as date, trip_distance
        FROM uber_trips
        WHERE date between '2013-07-01' AND '2013-07-31'
        )
    

"""

In [ ]:
QUERY_3 = """
SELECT
        trip_distance,
        PERCENT_RANK() OVER (ORDER BY trip_distance DESC) percent_rank
        
FROM (

        SELECT date(tpep_pickup_datetime) as date, trip_distance
        FROM taxi_trips
        
        UNION ALL
        SELECT date(pickup_datetime) as date, trip_distance
        FROM uber_trips
       
   
)
"""
#PERCENT_RANK() OVER (ORDER BY trip_distance DESC) percent_rank

In [ ]:
# TOBEDELETED Read Data
with connection:
    result = connection.execute(QUERY_3)

for row in result:
    print(row)

In [ ]:
write_query_to_file(QUERY_3, "question_3.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [19]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)